# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921742034266                   -0.69    5.9    364ms
  2   -7.926166419739       -2.35       -1.22    1.0    173ms
  3   -7.926839372456       -3.17       -2.37    1.9    205ms
  4   -7.926861215477       -4.66       -3.04    2.0    225ms
  5   -7.926861648306       -6.36       -3.39    1.9    225ms
  6   -7.926861668597       -7.69       -3.76    1.6    179ms
  7   -7.926861679073       -7.98       -4.12    1.2    168ms
  8   -7.926861681801       -8.56       -5.08    1.4    174ms
  9   -7.926861681862      -10.22       -5.22    3.4    234ms
 10   -7.926861681872      -10.98       -6.40    1.0    166ms
 11   -7.926861681873      -12.21       -6.59    2.9    330ms
 12   -7.926861681873      -14.10       -6.80    1.0    167ms
 13   -7.926861681873   +  -15.05       -7.55    1.0    165ms
 14   -7.926861681873      -15.05       -7.80    2.2    211ms
 15   -7.

If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921713745892                   -0.69    6.0    310ms
  2   -7.926166400953       -2.35       -1.22    1.0    173ms
  3   -7.926838613869       -3.17       -2.37    1.9    228ms
  4   -7.926861205574       -4.65       -3.02    2.2    246ms
  5   -7.926861640948       -6.36       -3.36    1.8    199ms
  6   -7.926861665656       -7.61       -3.71    1.6    218ms
  7   -7.926861679269       -7.87       -4.14    1.1    165ms
  8   -7.926861681801       -8.60       -5.17    1.9    179ms
  9   -7.926861681864      -10.20       -5.26    3.0    255ms
 10   -7.926861681872      -11.09       -6.41    1.0    465ms
 11   -7.926861681873      -12.41       -6.74    3.0    190ms
 12   -7.926861681873   +    -Inf       -7.42    1.0    146ms
 13   -7.926861681873   +    -Inf       -7.84    2.6    177ms
 14   -7.926861681873      -14.57       -9.08    1.2    154ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921734770008                   -0.69    5.9    252ms
  2   -7.926168381682       -2.35       -1.22    1.0    175ms
  3   -7.926842337868       -3.17       -2.37    1.9    183ms
  4   -7.926864573864       -4.65       -3.03    2.1    197ms
  5   -7.926865052644       -6.32       -3.36    2.0    178ms
  6   -7.926865076401       -7.62       -3.70    1.6    156ms
  7   -7.926865090131       -7.86       -4.11    1.1    149ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       
